In [1]:
# Import modules.
import os
import matplotlib.pyplot as mp

# Import tools for keras.
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from scipy         import stats

import datetime
import time

import copy
import json

from functools import reduce

import seaborn as sns

In [2]:
from imp import reload
import TensorFlowUtils
reload(TensorFlowUtils)
import SeisUtils
reload(SeisUtils)
import GAN
reload(GAN)
import ModelBase
reload(ModelBase)

Using TensorFlow backend.


<module 'ModelBase' from 'C:\\Users\\micha\\Documents\\CS101\\seis-GAN\\ModelBase.py'>

In [3]:
from TensorFlowUtils import show_graph
from SeisUtils       import extract_func, plot_wforms
from SeisUtils       import SeisData
from SeisUtils       import SeisGenerator
from GAN             import GAN
from ModelBase       import ModelBase

In [4]:
project_dir =  'GAN_2019-05-09-B' # 'GAN_2019-05-05' # 'GAN_2019-04-16-I' # 'GAN_2019-05-09-B' # 'GAN_2019-03-12-B' # 'GAN-2018-02-26-B' 'GAN_2019-03-01' # 
print('Found project directory "{}":'.format(project_dir), True)
config_path = os.path.join(project_dir, 'config.json')
config_d    = {}
with open(config_path, 'r') as config_file:
    config_d = json.load(config_file)
config_d['restore'] = True
config_d['data_path'] = "wformMat_jpm4p_181106_downsample-5x.h5"
config_d['load_data_into_memory'] = False
# config_d['restore_ckpt'] = 'ganckpt.ckpt-121233'
# config_d['train_frac'] = 0.74
# config_d['transform'] = 'from-' + config_d['transform'] if 'transform' in config_d else False
print('Loaded config "{}":\n'.format(config_path), json.dumps(config_d, indent = 4) )

Found project directory "GAN_2019-05-09-B": True
Loaded config "GAN_2019-05-09-B\config.json":
 {
    "data_path": "wformMat_jpm4p_181106_downsample-5x.h5",
    "data_format": "channels_last",
    "train_frac": 0.74,
    "burn_seconds": 2.5,
    "input_seconds": 80,
    "output_seconds": null,
    "measure_rate": 20,
    "normalize_data": true,
    "predict_normalization_factor": true,
    "batch_size": 256,
    "epochs": 200,
    "random_seed": 7,
    "load_data_into_memory": false,
    "metas": [
        "dist",
        "magn"
    ],
    "weight_loss": false,
    "lr_gen": 0.0001,
    "lr_dis": 0.0001,
    "conditional_config": {
        "one_hot_encode": true,
        "aux_classify": false
    },
    "bins_s": [
        10,
        [
            3,
            4,
            4.5,
            5,
            5.5,
            6,
            7,
            "inf"
        ]
    ],
    "num_channels": 3,
    "directory": "GAN_2019-05-09-B",
    "restore": true,
    "restore_ckpt": null,
  

In [5]:
class GANModel(ModelBase):
    
    def __init__(self, config):
        
        super(GANModel, self).__init__(config)
        
    def build_graph_and_session(self):
        
        self.graph = tf.Graph()
        with self.graph.as_default() as g:
            
            sessConfig        = tf.ConfigProto(allow_soft_placement = True)
            self.sess         = tf.Session(config = sessConfig)
            restore_path      = os.path.join(self.checkpoint_dir, self.config.get('restore_ckpt')) if self.config.get('restore_ckpt') else tf.train.latest_checkpoint(self.checkpoint_dir)
            self.saver        = tf.train.import_meta_graph(restore_path + '.meta')
            self.saver       .restore(self.sess, restore_path)

            # Get latent placeholder and output tensor to generator.
            self.training_bool_placeholder  = g.get_tensor_by_name('training_bool:0')            
            self.latent_placeholder         = g.get_tensor_by_name('latent:0')
            self.gen_output_tensor          = g.get_tensor_by_name('gen/output:0')

            # Get input placeholder and output tensor to discriminator. 
            self.dis_input_placeholder = g.get_tensor_by_name('dis/input:0') # Can be used for real or fake.
            self.dis_output_tensor     = g.get_tensor_by_name('dis/real/output:0') # @ to change
            
            # Get global step.
            self.global_step = tf.train.get_global_step()
                        
            # Setup conditional placeholder.
            if self.config.get('bins_s'):
                self.conditional_placeholders = [g.get_tensor_by_name('conditional:0'), g.get_tensor_by_name('conditional_1:0')]
            else:
                self.conditional_placeholders = None    
            
            # Setup predicted normalization tensor.
            if self.config.get('predict_normalization_factor'):
                self.pred_norm_fake_tensor = g.get_tensor_by_name('gen/pred_norm:0')
            else:
                self.pred_norm_fake_tensor = None

        
    #-------------------
    # Generator
    #-------------------

    np.random.seed(7)
    def latent(self, N = 4): 

        latent_shape = [N] + self.latent_placeholder.shape.as_list()[1:]

        latent = np.random.normal(size = latent_shape)
        return latent

    def generator(self, latent, training = False, conditional = None, **kwargs):
        
        feed_dict = {self.latent_placeholder : latent, self.training_bool_placeholder : training}
        if conditional is not None:
            feed_dict.update({cp : m for cp, m in zip(self.conditional_placeholders, conditional if isinstance(conditional, list) else [conditional])})
            
        if self.pred_norm_fake_tensor is not None:
            gen, pred_norm = self.sess.run([self.gen_output_tensor, self.pred_norm_fake_tensor], feed_dict = feed_dict)
            gen = gen * np.exp(pred_norm[:, None, :])
        else:
            gen = self.sess.run(self.gen_output_tensor, feed_dict = feed_dict)
        
        if self.config.get('normalizers'):
            gen = self.config['normalizers'][0] * gen
            
        if self.transformation is not None:
            gen = self.transformation.retransform(gen, **kwargs)
            
        return gen

    #-------------------
    # Discriminator.
    #-------------------

    def discriminator(self, input_real_or_fake):

        feed_dict = {self.dis_input_placeholder : input_real_or_fake, self.training_bool_placeholder : training}
        if conditional is not None:
            feed_dict.update({cp : m for cp, m in zip(self.conditional_placeholders, conditional if isinstance(conditional, list) else [conditional])})
            
        return self.sess.run(self.dis_output_tensor, feed_dict = feed_dict)



In [6]:
gm = GANModel(config_d)

config {'data_path': 'wformMat_jpm4p_181106_downsample-5x.h5', 'data_format': 'channels_last', 'train_frac': 0.74, 'burn_seconds': 2.5, 'input_seconds': 80, 'output_seconds': None, 'measure_rate': 20, 'normalize_data': True, 'predict_normalization_factor': True, 'batch_size': 256, 'epochs': 200, 'random_seed': 7, 'load_data_into_memory': False, 'metas': ['dist', 'magn'], 'weight_loss': False, 'lr_gen': 0.0001, 'lr_dis': 0.0001, 'conditional_config': {'one_hot_encode': True, 'aux_classify': False}, 'bins_s': [10, [3, 4, 4.5, 5, 5.5, 6, 7, 'inf']], 'num_channels': 3, 'directory': 'GAN_2019-05-09-B', 'restore': True, 'restore_ckpt': None, 'debug': True, 'model_notes': 'Increase size of fully connected layers.', 'normalizers': None}
Testing    Samples: 37666
Validation Samples: 30133
Training   Samples: 192965 

[(260764,), (260764,)]
[(11,), (8,)]
conditional_metav (shape): [(260764, 10), (260764, 7)]
weights: [0.6389864 1.238719  1.238719  ... 2.0003831 2.0003831 2.0003831]
   max  =  21

In [ ]:
#--------------------------------------
# Random Batch - for visual comparison
#--------------------------------------

# Save new example batch if desired.
if False:
    batch           = gm.SG_train.random_batch(normalize = False)
    x               = batch.x
    x_normalization = batch.x_normalization
    v               = batch.metav # Defaults to None
    np.save('example_batch_x.npy', x)
    np.save('example_batch_x_normalization.npy', x_normalization)
    for i, v_i in enumerate(v):
        np.save('example_batch_v_{}.npy'.format(i), v_i)
    
# Load new example batch.
x               = np.load('example_batch_x.npy')
x_normalization = np.load('example_batch_x_normalization.npy')
v               = []
for i in range(2):
    v.append(np.load('example_batch_v_{}.npy'.format(i)))

#----------------------------
# Generated Batch
#----------------------------

n        = 40 # Number of examples to show
N        = gm.batch_size
predgen  = gm.generator(gm.latent(N), training = False, conditional = v if v is not None else None)[0:n, ...]
m_steps  = gm.config.get('input_seconds', 20) * gm.config.get('measure_rate', 20)
realdata = x[0:n, 0:m_steps, :]
fig, axs = plot_wforms(
    np.vstack([predgen, realdata]),
    figsize = (30, np.floor(7.5*n)), 
    ylabels = ['gen' + str(i+1) for i in range(n)] + ['real' + str(i+1) for i in range(n)],
    xlim    = [0, gm.config.get('input_seconds', 20) * gm.config.get('measure_rate', 20)],
)

#-----------------------------
# Plot Examples
#-----------------------------
    
# predgen  = gm.generator(gm.latent(n), conditional = v[0:n, ...])
# fig, axs = plot_wforms(predgen, figsize = (30, 30) )
for i in range(n):
    for j in range(3):
        axs[i, j].get_lines()[0].set_c(plt.rcParams['axes.prop_cycle'].by_key()['color'][3])
                                   
fig.tight_layout() 
global_step = gm.sess.run(gm.global_step)
print('Saving image at global step:', global_step)
fig.savefig(os.path.join(project_dir, 'generated_wforms_at_step_{}_{}.png'.format(global_step, str(time.time()).split('.')[0])))

# Test Code

In [1276]:
1/0 # Intentional error to stop 'run all cells'.

ZeroDivisionError: division by zero